In [1]:
import time
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import matplotlib.pyplot as plt

## Dataset

In [2]:
train_dataset_path = '/content/drive/MyDrive/17Flowers/train'
width = height = 224
batch_size = 32

idg = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    brightness_range = (0.8,1.2),
    zoom_range = 0.1,
    shear_range = 0.3,
    rotation_range = 10,
    validation_split = 0.2
)

train_data = idg.flow_from_directory(
    train_dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "training"

)
val_data = idg.flow_from_directory(
    train_dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "validation"

)

Found 823 images belonging to 17 classes.
Found 197 images belonging to 17 classes.


## Model

In [3]:
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights = 'imagenet',
    input_shape=(width,height,3),
    pooling = 'avg',
)

model = tf.keras.models.Sequential([
        base_model,
        #Dropout(0.3),
        # Flatten(),
        # Dense(1024,activation='relu'),
        Dense(256,activation='relu'),
        Dense(17,activation='softmax')

])

9406464/9406464 [==============================] - 1s 0us/step


In [4]:
for layer in base_model.layers[:-2]:  #freeze layer
  layer.trainable = False

In [5]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [6]:
model.fit(train_data,validation_data=val_data,epochs=10)

Epoch 1/10
26/26 [==============================] - 781s 30s/step - loss: 1.6099 - accuracy: 0.5565 - val_loss: 0.7207 - val_accuracy: 0.7716
Epoch 2/10
26/26 [==============================] - 17s 647ms/step - loss: 0.3938 - accuracy: 0.9040 - val_loss: 0.4198 - val_accuracy: 0.8528
Epoch 3/10
26/26 [==============================] - 17s 657ms/step - loss: 0.2276 - accuracy: 0.9502 - val_loss: 0.4303 - val_accuracy: 0.8477
Epoch 4/10
26/26 [==============================] - 17s 655ms/step - loss: 0.1309 - accuracy: 0.9733 - val_loss: 0.3663 - val_accuracy: 0.8274
Epoch 5/10
26/26 [==============================] - 17s 650ms/step - loss: 0.0945 - accuracy: 0.9793 - val_loss: 0.3449 - val_accuracy: 0.8680
Epoch 6/10
26/26 [==============================] - 17s 666ms/step - loss: 0.0533 - accuracy: 0.9951 - val_loss: 0.2554 - val_accuracy: 0.8832
Epoch 7/10
26/26 [==============================] - 17s 655ms/step - loss: 0.0512 - accuracy: 0.9915 - val_loss: 0.2437 - val_accuracy: 0.9289


In [7]:
test_dataset_path = '/content/drive/MyDrive/17Flowers/test'
width=height = 224
batch_size = 32
idg = ImageDataGenerator(
    rescale = 1./255,
)


test_data = idg.flow_from_directory(
    test_dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
)

Found 340 images belonging to 17 classes.


In [8]:
model.evaluate(test_data)

11/11 [==============================] - 239s 24s/step - loss: 0.2515 - accuracy: 0.9265


[0.25147515535354614, 0.9264705777168274]

In [9]:
model.save('flowers_transfer_learning_1.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
